In [1]:
%load_ext autoreload
%load_ext autotime

time: 286 µs (started: 2024-10-09 16:48:51 -07:00)


In [2]:
import os
import sys
import time
import warnings

# Add WisIO to the path
sys.path.append(os.path.abspath("../"))

# Filter warnings
warnings.filterwarnings('ignore')

time: 960 µs (started: 2024-10-09 16:48:51 -07:00)


In [3]:
analyzer = "dftracer"
checkpoint = True
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = f".wisio/{analyzer}/{int(time.time())}"
bottleneck_dir = f"{run_dir}/bottlenecks"
checkpoint_dir = f"{run_dir}/checkpoints"
percentile = 0.95
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/cm1/APP/node-1/v1/RAW/*.pfw.gz"

time: 1.32 ms (started: 2024-10-09 16:48:51 -07:00)


In [4]:
# Parameters
analyzer = "dftracer"
checkpoint = True
checkpoint_dir = "/usr/workspace/iopp/wisio_logs/_checkpoints/dftracer-unet3d-dlio-v100-node-1-v1"
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = ".wisio/dftracer/unet3d-dlio-v100-node-1-v1"
percentile = 0.99
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/unet3d/dlio-v100/node-1/v1/RAW/*.pfw.gz"


time: 1.19 ms (started: 2024-10-09 16:48:52 -07:00)


In [5]:
from distributed import Client
from hydra import compose, initialize
from hydra.core.hydra_config import HydraConfig
from hydra.utils import instantiate
from wisio.config import init_hydra_config_store
from wisio.utils.file_utils import ensure_dir

ensure_dir(run_dir)

with initialize(version_base=None, config_path=None):
    init_hydra_config_store()
    cfg = compose(
        config_name="config",
        overrides=[
            f"+analyzer={analyzer}",
            f"analyzer.bottleneck_dir={bottleneck_dir}",
            f"analyzer.checkpoint={checkpoint}",
            f"analyzer.checkpoint_dir={checkpoint_dir}",
            f"hydra.run.dir={run_dir}",
            f"hydra.runtime.output_dir={run_dir}",
            f"logical_view_types={logical_view_types}",
            f"output.max_bottlenecks={output_max_bottlenecks}",
            f"output.root_only={output_root_only}",
            f"percentile={percentile}",
            f"trace_path={trace_path}",
        ],
        return_hydra_config=True,
    )
    HydraConfig.instance().set_config(cfg)
    cluster = instantiate(cfg.cluster)
    client = Client(cluster)
    analyzer = instantiate(cfg.analyzer)
    output = instantiate(cfg.output)

time: 6.3 s (started: 2024-10-09 16:31:03 -07:00)


In [6]:
result = analyzer.analyze_trace(
    trace_path=cfg.trace_path,
    # accuracy=cfg.accuracy,
    exclude_bottlenecks=cfg.exclude_bottlenecks,
    exclude_characteristics=cfg.exclude_characteristics,
    logical_view_types=cfg.logical_view_types,
    metrics=cfg.metrics,
    percentile=cfg.percentile,
    view_types=cfg.view_types,
)

time: 37.3 s (started: 2024-10-09 16:31:09 -07:00)


In [7]:
output.handle_result(metrics=cfg.metrics, result=result)

╭────────────────────────────────────────────── I/O Characteristics ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Runtime          145.67 seconds                                                                                │
│  I/O Time         3.22 seconds                                                                                  │
│                   ├── Read - 2.45 seconds (76.01%)                                                              │
│                   ├── Write - 0.00 seconds (0.04%)                                                              │
│                   └── Metadata - 2.17 seconds (67.31%)                                                          │
│  I/O Operations   1,101,599 ops                                                                                 │
│                   ├── Read - 43,368 ops (3.94%)                                                                 │
│                   ├── Write - 300 ops (0.03%)                                                                   │
│                   └── Metadata - 1,055,563 ops (95.82%)                                                         │
│  I/O Size         92.50 GiB                                                                                     │
│                   ├── Read - 92.50 GiB (100.00%)                                                                │
│                   └── Write - 20.39 kiB (0.00%)                                                                 │
│  Read Requests    4 kiB-4 MiB - 43,336 ops                                                                      │
│                   ├── <4 kiB - 11,927 ops (27.52%)                                                              │
│                   ├── 4-16 kiB - 2,145 ops (4.95%)                                                              │
│                   ├── 16-64 kiB - 2,704 ops (6.24%)                                                             │
│                   ├── 256 kiB-1 MiB - 4 ops (0.01%)                                                             │
│                   └── 1-4 MiB - 26,556 ops (61.28%)                                                             │
│  Write Requests   4 kiB-4 kiB - 300 ops                                                                         │
│                   └── <4 kiB - 300 ops (100.00%)                                                                │
│  Nodes            1 node                                                                                        │
│  Apps             1 app                                                                                         │
│  Processes/Ranks  96 processes                                                                                  │
│  Files            6,073 files                                                                                   │
│                   ├── Shared: 5,464 files (89.97%)                                                              │
│                   └── FPP: 609 files (10.03%)                                                                   │
│  Time Periods     68 time periods (Time Granularity: 1,000,000.0)                                               │
│  Access Pattern   Sequential: 43,668 ops (100.00%) - Random: 0 ops (0.00%)                                      │
│                                                                                                                 │
╰─ R: Read - W: Write - M: Metadata  ─────────────────────────────────────────────────────────────────────────────╯
                                                                                                                   
╭────────────────────────────────────── 75 I/O Bottlenecks with 209 Reasons ──────────────────────────────────────╮
│                                                       

time: 2.44 s (started: 2024-10-09 16:31:46 -07:00)
